In [245]:
## Import necessary modules
%load_ext autoreload
%autoreload 2

import os,sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator, AutoLocator, FormatStrFormatter, ScalarFormatter, MultipleLocator, FixedLocator
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
from itertools import tee

sys.path.append(os.path.abspath('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/pyplots/lib/'))
from ce_funclib import determine_kernzeit as dtkz
#from ce_plotlib import abs_plot

%matplotlib tk

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Deklarationen

In [3]:
arcpath='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/datenhalde/1458/' #Leseverzeichnis
neededcols=[0,1,2,3,5,12,21] # Liste der Spalten, die aus dem Excelfile gelesen werden sollen
new_colnames=['tstp','clls','ange','verb','tt','acw','lost'] # Namen, die spaeter auf die Spalten kommen
shitcols=('Verbindungszeit [hh:mm:ss]','Nachbearbeitungszeit [hh:mm:ss]')
converts={shitcols[0]:str,shitcols[1]:str}
ipynb_name='1458_allrounder'
global ipynb_name

## Funktionen

#### data

In [4]:
######## GET A LIST OF MATCHING .xls FILES FROM THE GIVEN DIRECTORY
def collectxlfiles(arcpath):
    xlfilelist=list()

    for xlfile in os.listdir(arcpath):
        if xlfile.startswith('1458_daily'):
            xlfileabs=os.path.join(arcpath,xlfile)
            xlfilelist.append(xlfileabs)
    return sorted(xlfilelist)

In [5]:
def turn_xls_to_df(file):

    excel_df=pd.read_excel(file,skiprows=3,skip_footer=1,usecols=neededcols,converters=converts)# die ersten 3 werden nicht benötigt, letzte auch nicht
    excel_df['Timestamp'] = pd.to_datetime(excel_df['Timestamp'], format=' %d.%m.%Y %H:%M ') # statt string soll das ein datetime werden
    excel_df.columns=new_colnames
    excel_df2=excel_df.set_index('tstp').copy() # die timestamps sollen der index sein
        
    return excel_df2

In [6]:
def add_averages(frame):

    frame['avtt']=(frame['tt']/frame['verb'])/60
    frame['avht']=(frame['ht']/frame['verb'])/60
    frame['avacw']=(frame['acw']/frame['verb'])/60
    colorder_with_abs=['clls','verb','lost','avht','avtt','avacw','ht','tt','acw']
    frame_with_abs=frame[colorder_with_abs].fillna(0)
    
    return frame_with_abs

In [7]:
def labelspucker(x_index):
    xx=x_index
    von_monat=xx.month.unique()[0]
    bis_monat=xx.month.unique()[-1]
    von_jahr=xx.year.unique()[0]
    bis_jahr=xx.year.unique()[-1]
    
    if (von_monat == bis_monat):
        labeltext=str(calendar.month_abbr[von_monat])+' '+str(von_jahr)
        
    return labeltext

#### plot

In [ ]:
def labelspucker(x_index):
    xx=x_index
    von_monat=xx.month.unique()[0]
    bis_monat=xx.month.unique()[-1]
    von_jahr=xx.year.unique()[0]
    bis_jahr=xx.year.unique()[-1]
    
    if (von_monat == bis_monat):
        labeltext=str(calendar.month_abbr[von_monat])+' '+str(von_jahr)
        
    return labeltext


def abs_plot(frame):
    gcol="#777777"
    lostcol="#AC003A"
    verbcol="#008EC4"
    ncol="#000000"
    kcol="#4F7DFF"
    
    fig,ax=plt.subplots()

    sonn = WeekdayLocator(6)
    weekFormatter = DateFormatter('%a %b %d')

    ax.xaxis.set_major_locator(sonn)
    ax.xaxis.set_major_formatter(weekFormatter)

    ax.grid(which='major', axis='y', linestyle=':', zorder=0)
    ax.set_axisbelow(True)

    d_ix=frame.index.get_level_values(0).unique()
    g_val=frame.xs('g', level='bz')['clls']
    k_val=frame.xs('k', level='bz')['clls']
    n_val=frame.xs('n', level='bz')['clls']
    verb=frame.xs('g', level='bz')['verb']
    lost=frame.xs('g', level='bz')['lost']

    bar_g=ax.bar(d_ix,g_val,color=gcol,width=0.8, label='calls')
    bar_n=ax.bar(d_ix,n_val, color=ncol, width=0.4, align='edge',label='davon nebenzeit')
    bar_verb=ax.bar(d_ix,verb,color=verbcol,width=-0.3, label='verbunden')
    bar_lost=ax.bar(d_ix,lost,color=lostcol,width=-0.3, bottom=verb,label='verloren')
    #bar_k=ax.bar(d_ix,k_val, color=kcol, width=0.2, align='center')

    ax.set_xlabel(labelspucker(d_ix))
    ax.set_ylabel('calls',rotation=90)
    
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels)
    
    ax.text(0.85, -0.1, 'abs_plot()', horizontalalignment='left', verticalalignment='bottom', transform=ax.transAxes, color='#A3ABBD', fontsize='6')

    fig.show()

In [416]:
def plotby_avhour(frame,vonbis):

    gcol,lostcol,verbcol,ncol,kcol,titcol,bgcol="#777777","#AC003A","#008EC4","#000000","#4F7DFF","#0084E0","#F4F4F4"

    fig,ax1 = plt.subplots()
    ax2=ax1.twiny()
    
    ax1.set_facecolor(bgcol)
    ax1.grid(which='major', axis='y', linestyle=':', zorder=0)
    ax1.grid(which='major', axis='x', linestyle='--', zorder=0, color=titcol, alpha=0.1)
    ax1.set_axisbelow(True)
    
    ax1.set_xlabel('Uhrzeit')
    ax1.set_ylabel('Calls')

    x_x=frame.index.values
    ax1.xaxis.set_major_locator(FixedLocator(x_x))
    ax1labels=["{:}:00".format(x) for x in x_x]; 
    ax1.set_xticklabels(ax1labels,fontsize='7', rotation='45');

    clls, verb, lost = frame['clls'], frame['verb'], frame['lost']

    cbar=ax1.bar(x_x,clls,width=0.8,color=gcol,label='calls')
    vbar=ax1.bar(x_x,verb,width=0.4,color=verbcol,label='verbunden')
    lbar=ax1.bar(x_x,lost,width=0.4,color=lostcol,bottom=verb,label='verloren')

    annahmequote=(frame['verb']/frame['clls'])*100

    ax2labels=["{0:.1f}%".format(x) for x in annahmequote.values];   ## muss so formatiert werden, siehe hier https://stackoverflow.com/questions/45056579/is-it-possible-to-format-the-labels-using-set-xticklabels-in-matplotlib
    ax2.set_xlim(ax1.get_xlim());
    ax2.set_xticks(annahmequote.index.values);
    ax2.set_xticklabels(ax2labels,fontsize='6.25',rotation='45',ha='left');

    oberer_rand=ax1.get_ylim()[1]
    for i in x_x:
        txt=(int(frame.loc[i,['lost']]))
        ax1.text(i,(oberer_rand*0.9725),txt,fontsize=6,horizontalalignment='center', color=lostcol)  
    
    von,bis=vonbis[0].strftime('%d.%m.%y'),vonbis[1].strftime('%d.%m.%y')
    fig.suptitle('1458: Calls per Stunde dargestellt von '+von+' bis '+bis,fontsize=9,color=titcol)
    
    plt.subplots_adjust(top=0.825, bottom=0.125)
    handles, labels = ax1.get_legend_handles_labels()
    
    labels[0],labels[1],labels[2]=str(labels[0]+' '+str(frame['clls'].sum())),str(labels[1]+' '+str(frame['verb'].sum())),str(labels[2]+' '+str(frame['lost'].sum()))
    
    fig.legend(handles,labels,loc='center', fontsize=7, bbox_to_anchor=(0.25,0.7),ncol=1)

## Datensammeln und Frame mit allen Daten herstellen

In [8]:
xlfilelist=collectxlfiles(arcpath)

In [9]:
greatframe=pd.DataFrame() # leeren df initialisieren
for i in xlfilelist:
    i_frame=turn_xls_to_df(i)
    greatframe=greatframe.append(i_frame) 
# alle files werden ordentlich in df konvertiert und an den ausserhalb der funktion kreierten df angehangen
# greatframe # hier sind alle daten vollständig enthalten, die gebraucht werden, der Ur-Frame

In [10]:
## Anrufzeiten statt datetime oder string nach timedelta (Sekunden) gewandelt
greatframe[['tt','acw']]=greatframe[['tt','acw']].apply(pd.to_timedelta).astype('timedelta64[s]')
greatframe['ht'] = (greatframe['tt']+greatframe['acw'])

In [11]:
## add kern- und nebenzeit column
greatframe['bz'] = greatframe.index.map(dtkz)

## Gruppierung nach Tagen (Datum)

In [12]:
daily_frame=greatframe.groupby([pd.Grouper(freq='D'),pd.Grouper('bz')]).sum().copy() # Nebenzeit nach Tagen gruppiert
daily_frame=daily_frame.unstack('bz').fillna(0).stack('bz')

In [13]:
g = daily_frame.sum(level=['tstp']) # einen Zwischenframe erstellen, der nur die Kern- und Nebenzeit summiert
g.assign(bz='g').set_index('bz', append=True).sort_index() # die Summe dem alten Frame beigeben
daily=daily_frame.append(g.assign(bz='g').set_index('bz', append=True)).sort_index() # die Summe dem alten Frame beigeben

In [14]:
daily_av=add_averages(daily)  # mit durschnittlichen Zeiten acw/calls etc.

### Slicing und Plotting für Tage (Datum)

In [15]:
d_ix=daily_av.index.get_level_values(0)  # das ist der datetime-index (level 0), kann mit monat,jahr usw. bedient werden

In [18]:
plotyear=2017
for i in range(11,12):
    plotmonth=daily_av.loc[(d_ix.month == i) & (d_ix.year == plotyear)]
    abs_plot(plotmonth)

## Gruppierung nach Wochentagen

In [496]:
## Zeitraum für den plot wählen und filtern
von_bis=(datetime.date(2018,3,1),datetime.date(2018,3,31))

In [497]:
zeitraum=greatframe.loc[(greatframe.index.date >= von_bis[0]) & (greatframe.index.date <= von_bis[1])]
von_bis_real=(zeitraum.index.date.min(),zeitraum.index.date.max())
hh_gruppiert=add_averages(zeitraum.groupby(zeitraum.index.hour).sum())  # nach stunden zusammengefasst

In [498]:
plotby_avhour(hh_gruppiert,von_bis_real)

#### kern- und nebenzeit in 2 subplots

In [479]:
ke,ne=zeitraum.loc[zeitraum['bz'] == 'k'],zeitraum.loc[zeitraum['bz'] == 'n']
ges_grp=add_averages(zeitraum.groupby(zeitraum.index.hour).sum())
hourindex=ges_grp.index

ke_grp=add_averages(ke.groupby(ke.index.hour).sum()).reindex(hourindex).fillna(0)
ne_grp=add_averages(ne.groupby(ne.index.hour).sum()).reindex(hourindex).fillna(0)

In [490]:
def plotby_avhour2(frameindex,framek,framen,vonbis):

    gcol,lostcol,verbcol,ncol,kcol,titcol,bgcol="#777777","#AC003A","#008EC4","#000000","#4F7DFF","#0084E0","#F4F4F4"
    fig = plt.figure()
    
    ###############################################
    ##### subplot nur für kernzeit################
    ###############################################
    ax1 = fig.add_subplot(121)
    ax2=ax1.twiny()
    
    ax1.set_facecolor(bgcol)
    ax1.grid(which='major', axis='y', linestyle=':', zorder=0)
    ax1.grid(which='major', axis='x', linestyle='--', zorder=0, color=titcol, alpha=0.1)
    ax1.set_axisbelow(True)
    
    ax1.set_xlabel('Kernzeit')
    ax1.set_ylabel('Calls')

    x_x=frameindex.values
    ax1.xaxis.set_major_locator(FixedLocator(x_x))
    ax1labels=["{:}:00".format(x) for x in x_x]; 
    ax1.set_xticklabels(ax1labels,fontsize='7', rotation='45');

    cllsk, verbk, lostk = framek['clls'], framek['verb'], framek['lost']

    cbark=ax1.bar(x_x,cllsk,width=0.8,color=gcol,label='calls')
    vbark=ax1.bar(x_x,verbk,width=0.4,color=verbcol,label='verbunden')
    lbark=ax1.bar(x_x,lostk,width=0.4,color=lostcol,bottom=verbk,label='verloren')

    annahmequotek=(framek['verb']/framek['clls'])*100

    ax2labels=["{0:.1f}%".format(x) for x in annahmequotek.values];   ## muss so formatiert werden, siehe hier https://stackoverflow.com/questions/45056579/is-it-possible-to-format-the-labels-using-set-xticklabels-in-matplotlib
    ax2.set_xlim(ax1.get_xlim());
    ax2.set_xticks(annahmequotek.index.values);
    ax2.set_xticklabels(ax2labels,fontsize='6.25',rotation='45',ha='left');

    oberer_rand=ax1.get_ylim()[1]
    for i in x_x:
        txt=(int(framek.loc[i,['lost']]))
        ax1.text(i,(oberer_rand*0.9725),txt,fontsize=6,horizontalalignment='center', color=lostcol)  
    
    
    ###############################################
    ##### subplot nur für nebenzeit################
    ###############################################
    
    ax_n1 = fig.add_subplot(122)
    ax_n2=ax_n1.twiny()
    
    ax_n1.set_facecolor(bgcol)
    ax_n1.grid(which='major', axis='y', linestyle=':', zorder=0)
    ax_n1.grid(which='major', axis='x', linestyle='--', zorder=0, color=titcol, alpha=0.1)
    ax_n1.set_axisbelow(True)
    
    ax_n1.set_xlabel('Nebenzeit')
    ax_n1.set_ylabel('Calls')

    x_x=frame.index.values
    ax_n1.xaxis.set_major_locator(FixedLocator(x_x))
    ax_n1labels=["{:}:00".format(x) for x in x_x]; 
    ax_n1.set_xticklabels(ax1labels,fontsize='7', rotation='45');

    cllsn, verbn, lostn = framen['clls'], framen['verb'], framen['lost'] # gibts schon
    
    cbar_n=ax_n1.bar(x_x,cllsn,width=0.8,color=gcol,label='calls')
    vbar_n=ax_n1.bar(x_x,verbn,width=0.4,color=verbcol,label='verbunden')
    lbar_n=ax_n1.bar(x_x,lostn,width=0.4,color=lostcol,bottom=verbn,label='verloren')
    
    annahmequoten=(framen['verb']/framen['clls'])*100
    
    ax_n2labels=["{0:.1f}%".format(x) for x in annahmequoten.values];   ## muss so formatiert werden, siehe hier https://stackoverflow.com/questions/45056579/is-it-possible-to-format-the-labels-using-set-xticklabels-in-matplotlib
    ax_n2.set_xlim(ax_n1.get_xlim());
    ax_n2.set_xticks(annahmequoten.index.values);
    ax_n2.set_xticklabels(ax_n2labels,fontsize='6.25',rotation='45',ha='left');

    oberer_rand=ax_n1.get_ylim()[1]
    for i in x_x:
        txt=(int(framen.loc[i,['lost']]))
        ax_n1.text(i,(oberer_rand*0.9725),txt,fontsize=6,horizontalalignment='center', color=lostcol)  
    
    ################################################
    ############ LEGENDE ###########################
    ################################################
    
    von,bis=vonbis[0].strftime('%d.%m.%y'),vonbis[1].strftime('%d.%m.%y')
    fig.suptitle('1458: Calls per Stunde dargestellt von '+von+' bis '+bis,fontsize=9,color=titcol)
    
    plt.subplots_adjust(top=0.825, bottom=0.125)
    khandles, klabels = ax1.get_legend_handles_labels()
    nhandles, nlabels = ax_n1.get_legend_handles_labels()
    
    klabels[0],klabels[1],klabels[2]=str(klabels[0]+' '+str(framek['clls'].sum())),str(klabels[1]+' '+str(framek['verb'].sum())),str(klabels[2]+' '+str(framek['lost'].sum()))
    nlabels[0],nlabels[1],nlabels[2]=str(nlabels[0]+' '+str(framen['clls'].sum())),str(nlabels[1]+' '+str(framen['verb'].sum())),str(nlabels[2]+' '+str(framen['lost'].sum()))
    
    
    ax1.legend(khandles,klabels,loc='center', fontsize=7, bbox_to_anchor=(0.25,0.7),ncol=1)
    ax_n1.legend(nhandles,nlabels,loc='center', fontsize=7, bbox_to_anchor=(0.25,0.7),ncol=1)

In [495]:
plotby_avhour2(hourindex,ke_grp,ne_grp,von_bis_real)

In [492]:
ke_grp

,clls,verb,lost,avht,avtt,avacw,ht,tt,acw
tstp,,,,,,,,,
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
5,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
6,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
7,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
8,117.0,108.0,8.0,3.395525,2.866204,0.529321,22003.0,18573.0,3430.0


In [493]:
ne_grp

,clls,verb,lost,avht,avtt,avacw,ht,tt,acw
tstp,,,,,,,,,
0,0,0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,0,0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,0,0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,0,0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,0,0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0
5,0,0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0
6,6,6,0,1.072222,0.783333,0.288889,386.0,282.0,104.0
7,31,31,0,1.646237,1.286559,0.359677,3062.0,2393.0,669.0
8,0,0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0
